In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1884497,2021-11-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1884498,2021-11-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1884499,2021-11-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1884500,2021-11-30,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
48903,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
48905,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
48907,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
48909,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
48911,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
1145796,2021-11-27,Arkansas,Arkansas,5001,3539,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1145798,2021-11-28,Arkansas,Arkansas,5001,3540,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1145800,2021-11-29,Arkansas,Arkansas,5001,3536,60.00,5000,Arkansas,AR,Arkansas,State,3017804
1145802,2021-11-30,Arkansas,Arkansas,5001,3539,60.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1884497,2021-11-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1884498,2021-11-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1884499,2021-11-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1884500,2021-11-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-12-01') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
680,2021-12-01,Snohomish,Washington,72145,797.00,53061,WA,County,822083,8775.88,96.95
1358,2021-12-01,Cook,Illinois,678881,12056.00,17031,IL,County,5150233,13181.56,234.09
2035,2021-12-01,Orange,California,334626,5736.00,6059,CA,County,3175692,10537.10,180.62
2711,2021-12-01,Maricopa,Arizona,802164,12801.00,4013,AZ,County,4485414,17883.83,285.39
3387,2021-12-01,Los Angeles,California,1529290,27184.00,6037,CA,County,10039107,15233.33,270.78
...,...,...,...,...,...,...,...,...,...,...,...
1882965,2021-12-01,Wheeler,Oregon,114,1.00,41069,OR,County,1332,8558.56,75.08
1883380,2021-12-01,King,Texas,22,0.00,48269,TX,County,272,8088.24,0.00
1883764,2021-12-01,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10
1884144,2021-12-01,Loving,Texas,9,0.00,48301,TX,County,169,5325.44,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
680,2021-12-01,Snohomish,Washington,72145,797.00,53061,WA,County,822083,8775.88,96.95,96.95,8775.88
1358,2021-12-01,Cook,Illinois,678881,12056.00,17031,IL,County,5150233,13181.56,234.09,234.09,13181.56
2035,2021-12-01,Orange,California,334626,5736.00,6059,CA,County,3175692,10537.10,180.62,180.62,10537.10
2711,2021-12-01,Maricopa,Arizona,802164,12801.00,4013,AZ,County,4485414,17883.83,285.39,285.39,17883.83
3387,2021-12-01,Los Angeles,California,1529290,27184.00,6037,CA,County,10039107,15233.33,270.78,270.78,15233.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1882965,2021-12-01,Wheeler,Oregon,114,1.00,41069,OR,County,1332,8558.56,75.08,75.08,8558.56
1883380,2021-12-01,King,Texas,22,0.00,48269,TX,County,272,8088.24,0.00,0.00,8088.24
1883764,2021-12-01,Esmeralda,Nevada,62,2.00,32009,NV,County,873,7101.95,229.10,229.10,7101.95
1884144,2021-12-01,Loving,Texas,9,0.00,48301,TX,County,169,5325.44,0.00,0.00,5325.44


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
1198519,2021-12-01,Hale,Alabama,3186,89.00,1065,AL,County,14651,21745.96,607.47,607.47,21745.96,1
1305539,2021-12-01,Clarke,Alabama,4853,86.00,1025,AL,County,23622,20544.41,364.07,364.07,20544.41,2
1197907,2021-12-01,Fayette,Alabama,3308,85.00,1057,AL,County,16302,20291.99,521.41,521.41,20291.99,3
695166,2021-12-01,Franklin,Alabama,6347,108.00,1059,AL,County,31362,20237.87,344.37,344.37,20237.87,4
1011228,2021-12-01,Winston,Alabama,4779,110.00,1133,AL,County,23629,20225.15,465.53,465.53,20225.15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1143351,2021-12-01,Converse,Wyoming,2328,42.00,56009,WY,County,13822,16842.71,303.86,303.86,16842.71,19
1008153,2021-12-01,Johnson,Wyoming,1358,16.00,56019,WY,County,8445,16080.52,189.46,189.46,16080.52,20
1143965,2021-12-01,Sublette,Wyoming,1491,23.00,56035,WY,County,9831,15166.31,233.95,233.95,15166.31,21
1424296,2021-12-01,Lincoln,Wyoming,2975,24.00,56023,WY,County,19830,15002.52,121.03,121.03,15002.52,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
936081,2021-12-01,Lowndes,Alabama,1782,68.00,1085,AL,County,9726,18322.02,699.16,699.16,18322.02,25,1
1198519,2021-12-01,Hale,Alabama,3186,89.00,1065,AL,County,14651,21745.96,607.47,607.47,21745.96,1,2
396185,2021-12-01,Walker,Alabama,12081,380.00,1127,AL,County,63521,19018.91,598.23,598.23,19018.91,13,3
828729,2021-12-01,Dallas,Alabama,5296,210.00,1047,AL,County,37196,14238.09,564.58,564.58,14238.09,62,4
934849,2021-12-01,Crenshaw,Alabama,2613,77.00,1041,AL,County,13772,18973.28,559.11,559.11,18973.28,15,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
128603,2021-12-01,Sheridan,Wyoming,5852,57.00,56033,WY,County,30485,19196.33,186.98,186.98,19196.33,11,19
1388424,2021-12-01,Uinta,Wyoming,4055,31.00,56041,WY,County,20226,20048.45,153.27,153.27,20048.45,8,20
1424296,2021-12-01,Lincoln,Wyoming,2975,24.00,56023,WY,County,19830,15002.52,121.03,121.03,15002.52,22,21
932385,2021-12-01,Albany,Wyoming,6836,39.00,56001,WY,County,38880,17582.30,100.31,100.31,17582.30,17,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,96.95,8775.88,30,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1880136,2021-11-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1880137,2021-11-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1880138,2021-11-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5
1880139,2021-11-30,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,5


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa[ '14day_case_avg' ] = usa.case_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg
1193547,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21745.96,2,1,1.00,nan
1193548,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21745.96,2,1,0.00,nan
1193549,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21745.96,2,1,0.00,nan
1193550,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,607.47,21745.96,2,1,0.00,nan
1193551,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,607.47,21745.96,2,1,2.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448460,2021-11-27,Crook,Wyoming,984,20.00,56011,WY,County,7584,12974.68,263.71,276.90,13172.47,10,23,0.00,2.00
1448461,2021-11-28,Crook,Wyoming,984,20.00,56011,WY,County,7584,12974.68,263.71,276.90,13172.47,10,23,0.00,2.00
1448462,2021-11-29,Crook,Wyoming,993,20.00,56011,WY,County,7584,13093.35,263.71,276.90,13172.47,10,23,9.00,2.36
1448463,2021-11-30,Crook,Wyoming,995,21.00,56011,WY,County,7584,13119.73,276.90,276.90,13172.47,10,23,2.00,2.14


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa[ '14day_death_avg' ] = usa.death_day.rolling(14).mean()
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
932333,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,25,1.00,3.93,0.00,nan
932334,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,25,0.00,3.29,0.00,nan
932335,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,25,0.00,2.79,0.00,nan
932336,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,25,0.00,2.71,0.00,nan
932337,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,699.16,18322.02,1,25,0.00,2.57,0.00,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391796,2021-11-27,Teton,Wyoming,5366,14.00,56039,WY,County,23464,22869.08,59.67,59.67,22975.62,23,3,0.00,4.64,0.00,0.00
391797,2021-11-28,Teton,Wyoming,5366,14.00,56039,WY,County,23464,22869.08,59.67,59.67,22975.62,23,3,0.00,4.64,0.00,0.00
391798,2021-11-29,Teton,Wyoming,5377,14.00,56039,WY,County,23464,22915.96,59.67,59.67,22975.62,23,3,11.00,4.86,0.00,0.00
391799,2021-11-30,Teton,Wyoming,5390,14.00,56039,WY,County,23464,22971.36,59.67,59.67,22975.62,23,3,13.00,4.79,0.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-12-01') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
473405,2021-12-01,Imperial,California,38516,772.00,6025,CA,County,181215,21254.31,426.01,426.01,21254.31,1,3,39.00,66.50,0.00,0.50
234877,2021-12-01,San Bernardino,California,374855,5967.00,6071,CA,County,2180085,17194.51,273.70,273.70,17194.51,2,6,312.00,407.57,5.00,4.71
3387,2021-12-01,Los Angeles,California,1529290,27184.00,6037,CA,County,10039107,15233.33,270.78,270.78,15233.33,3,14,1454.00,1021.71,18.00,18.64
1144508,2021-12-01,Tuolumne,California,7877,144.00,6109,CA,County,54478,14459.05,264.33,264.33,14459.05,4,19,11.00,11.21,3.00,0.43
104621,2021-12-01,Stanislaus,California,91620,1422.00,6099,CA,County,550660,16638.22,258.24,258.24,16638.22,5,8,72.00,99.21,5.00,1.50
831766,2021-12-01,Inyo,California,2568,46.00,6027,CA,County,18039,14235.82,255.00,255.00,14235.82,6,21,2.00,5.86,0.00,0.29
72375,2021-12-01,Shasta,California,26113,458.00,6089,CA,County,180080,14500.78,254.33,254.33,14500.78,7,18,32.00,37.86,7.00,3.00
753297,2021-12-01,Merced,California,45539,674.00,6047,CA,County,277680,16399.81,242.73,242.73,16399.81,8,9,73.00,53.07,3.00,1.29
83135,2021-12-01,San Joaquin,California,107604,1837.00,6077,CA,County,762148,14118.52,241.03,241.03,14118.52,9,22,45.00,100.07,0.00,0.36
1009940,2021-12-01,Kings,California,35221,365.00,6031,CA,County,152940,23029.29,238.66,238.66,23029.29,10,2,15.00,34.71,0.00,1.07


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
1775770,2021-12-01,Lassen,California,7967,58.00,6035,CA,County,30573,26058.94,189.71,189.71,26058.94,19,1,5.00,6.57,0.00,0.29
1009940,2021-12-01,Kings,California,35221,365.00,6031,CA,County,152940,23029.29,238.66,238.66,23029.29,10,2,15.00,34.71,0.00,1.07
473405,2021-12-01,Imperial,California,38516,772.00,6025,CA,County,181215,21254.31,426.01,426.01,21254.31,1,3,39.00,66.50,0.00,0.50
129858,2021-12-01,Tulare,California,86137,1099.00,6107,CA,County,466195,18476.60,235.74,235.74,18476.60,11,4,50.00,89.57,3.00,4.29
295641,2021-12-01,Kern,California,157875,1804.00,6029,CA,County,900202,17537.73,200.40,200.40,17537.73,14,5,213.00,212.79,0.00,4.50
234877,2021-12-01,San Bernardino,California,374855,5967.00,6071,CA,County,2180085,17194.51,273.70,273.70,17194.51,2,6,312.00,407.57,5.00,4.71
47632,2021-12-01,Madera,California,26233,311.00,6039,CA,County,157327,16674.19,197.68,197.68,16674.19,16,7,39.00,36.21,0.00,1.29
104621,2021-12-01,Stanislaus,California,91620,1422.00,6099,CA,County,550660,16638.22,258.24,258.24,16638.22,5,8,72.00,99.21,5.00,1.50
753297,2021-12-01,Merced,California,45539,674.00,6047,CA,County,277680,16399.81,242.73,242.73,16399.81,8,9,73.00,53.07,3.00,1.29
1079431,2021-12-01,Glenn,California,4574,51.00,6021,CA,County,28393,16109.60,179.62,179.62,16109.60,21,10,0.00,2.64,0.00,0.29


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,14day_case_avg,death_day,14day_death_avg
473405,2021-12-01,Imperial,California,38516,772.00,6025,CA,County,181215,21254.31,426.01,426.01,21254.31,1,3,39.00,66.50,0.00,0.50
234877,2021-12-01,San Bernardino,California,374855,5967.00,6071,CA,County,2180085,17194.51,273.70,273.70,17194.51,2,6,312.00,407.57,5.00,4.71
3387,2021-12-01,Los Angeles,California,1529290,27184.00,6037,CA,County,10039107,15233.33,270.78,270.78,15233.33,3,14,1454.00,1021.71,18.00,18.64
1144508,2021-12-01,Tuolumne,California,7877,144.00,6109,CA,County,54478,14459.05,264.33,264.33,14459.05,4,19,11.00,11.21,3.00,0.43
104621,2021-12-01,Stanislaus,California,91620,1422.00,6099,CA,County,550660,16638.22,258.24,258.24,16638.22,5,8,72.00,99.21,5.00,1.50
831766,2021-12-01,Inyo,California,2568,46.00,6027,CA,County,18039,14235.82,255.00,255.00,14235.82,6,21,2.00,5.86,0.00,0.29
72375,2021-12-01,Shasta,California,26113,458.00,6089,CA,County,180080,14500.78,254.33,254.33,14500.78,7,18,32.00,37.86,7.00,3.00
753297,2021-12-01,Merced,California,45539,674.00,6047,CA,County,277680,16399.81,242.73,242.73,16399.81,8,9,73.00,53.07,3.00,1.29
83135,2021-12-01,San Joaquin,California,107604,1837.00,6077,CA,County,762148,14118.52,241.03,241.03,14118.52,9,22,45.00,100.07,0.00,0.36
1009940,2021-12-01,Kings,California,35221,365.00,6031,CA,County,152940,23029.29,238.66,238.66,23029.29,10,2,15.00,34.71,0.00,1.07


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'current_crate', 'state_case', 'case_day', '14day_case_avg']]
top10_ca_case_df

,date,county,cases,current_crate,state_case,case_day,14day_case_avg
0,03/20/20,Imperial,4,2.21,3,4.00,29.21
1,03/21/20,Imperial,4,2.21,3,0.00,25.57
2,03/22/20,Imperial,4,2.21,3,0.00,25.57
3,03/23/20,Imperial,4,2.21,3,0.00,25.57
4,03/24/20,Imperial,9,4.97,3,5.00,17.14
...,...,...,...,...,...,...,...
6170,11/27/21,Glenn,4551,16028.60,10,0.00,1.93
6171,11/28/21,Glenn,4551,16028.60,10,0.00,1.93
6172,11/29/21,Glenn,4566,16081.43,10,15.00,2.50
6173,11/30/21,Glenn,4574,16109.60,10,8.00,2.71


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','deaths', 'current_drate', 'state_death', 'death_day', '14day_death_avg']]
top10_ca_death_df

,date,county,deaths,current_drate,state_death,death_day,14day_death_avg
0,03/20/20,Imperial,0.00,0.00,1,0.00,0.00
1,03/21/20,Imperial,0.00,0.00,1,0.00,0.00
2,03/22/20,Imperial,0.00,0.00,1,0.00,0.00
3,03/23/20,Imperial,0.00,0.00,1,0.00,0.00
4,03/24/20,Imperial,0.00,0.00,1,0.00,0.00
...,...,...,...,...,...,...,...
6279,11/27/21,Kings,363.00,237.35,10,0.00,1.14
6280,11/28/21,Kings,363.00,237.35,10,0.00,1.14
6281,11/29/21,Kings,364.00,238.00,10,1.00,1.07
6282,11/30/21,Kings,365.00,238.66,10,1.00,1.21


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)